In [ ]:
import scipy
import sys
from lxml import etree
import FeatureExtraction as FE
sys.path.insert(0, '../notebooks/libs/')

In [ ]:
#Jedna linija je jedan dokument.
#Lista svih linija.
#id - autor, broj njegove linije

pathToFile='../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
pathToIds = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'
tree=etree.parse(pathToFile)
predators_ids = FE.sexual_predator_ids(pathToIds)

In [ ]:
#path_to_file - put do xml datoteke
#predator_ids - lista id seksualnih predatora
def prepare_data_for_clustering(path_to_file,predators_ids):
    tree = etree.parse(pathToFile)
    dictionary = FE.extract_author_conversation_node_dictionary_from_XML(tree)
    #lista konverzacija u kojima sudjeluje predator
    conversations = []
    conv_ids = []
    line_nrs = []
    line_texts = []
    for predator in predators_ids:
        convs = dictionary[predator]
        for c in convs:
            conversations.append(c)
    for conv_node in conversations:
        conv_id  = conv_node.attrib["id"]
        messgs = conv_node.xpath("./message")
        for mess_node in messgs:          
            text = mess_node.xpath("./text")[0].text
            if text is None:
                continue
            conv_ids.append(conv_id)
            line_nr = mess_node.attrib["line"]
            line_nrs.append(line_nr)
            line_texts.append(text)
    return conv_ids,line_nrs,line_texts

In [ ]:
conv_ids,line_nrs,line_texts = prepare_data_for_clustering(pathToFile,predators_ids)

In [ ]:
print len(conv_ids)
print conv_ids[0:10]
print len(line_nrs)
print line_nrs[0:10]
print len(line_texts)
print line_texts[0:10]

In [ ]:
cnt=0
for l in line_texts:
    if l is None:
        cnt+=1
print cnt

message_node?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def make_tf_idf_matrix_for_clustering_from_XML(line_texts):
    tfidf=TfidfVectorizer(stop_words='english',min_df=10)
    return tfidf.fit_transform(line_texts), tfidf.get_feature_names()

In [ ]:
matrix, ft_names = make_tf_idf_matrix_for_clustering_from_XML(line_texts)

In [ ]:
matrix.shape

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
random_state = 170
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(matrix)

In [ ]:
#ako je flag true, u file ce se ispisati i sadrzaj recenica
#ako je flag false, file je u formatu id konverzacije, broj linije
import codecs
def dump_prediction_to_file(prediction_vector,conv_ids,line_nrs,line_texts,show_text=True):
    f = codecs.open('class_zero','w','utf-8')
    g = codecs.open('class_one','w','utf-8')
    for i,p in enumerate(prediction_vector):
        if p==0:
            line = str(conv_ids[i])+" "+str(line_nrs[i])
            if show_text:
                line+=" "+line_texts[i]+"\n"
            else:
                line+="\n"
            f.write(line)
        else:
            line = str(conv_ids[i])+" "+str(line_nrs[i])
            if show_text:
                line+=" "+line_texts[i]+"\n"
            else:
                line+="\n"
            g.write(line)
            
    f.close()
    g.close()
    

In [ ]:
dump_prediction_to_file(y_pred,conv_ids,line_nrs,line_texts,show_text=False)